# Peforth magics %ai and %chat

In [1]:
_file_ = "Jupyter Notebooks/OpenAI/ChatLKK/develop/peforth AI magics.ipynb"

## 祭出 peforth 
peforth 有 %f, %ai 以及 %chat 三個 Jupyter notebook 的 magics.

|Command|Description|
|---|:---|
|**%f**|FORTH statement, 例如： ." Hello World!", help 等等。任意組合 commands, 管理大量 commands 的交談式語言，非常精簡、優美。|
|**%ai**|對 AI 發問，不帶歷史。例如： Tell me a joke.|
|**%chat**|對 AI 發問，帶上歷史。例如： Why is it funny ?|


In [2]:
import os, peforth
from IPython.display import display, Markdown
%f display constant display // ( -- obj ) Jupyternotebook display function
%f Markdown constant Markdown // ( -- obj ) Jupyternotebook Markdown class
display(Markdown("## 祭出 Columbus"))

## 祭出 Columbus

In [3]:
from columbus_api import Columbus
columbus = Columbus()

## 準備工作，為 AI 提供必要零件。
從 columbus_api 取得 llm. 這個例子是取用 LLM for LangChain, 如果是別的，例如 LLM for CrewAI, LLM for DSPy 則以下的 llm_wrapper 會稍有不同。

In [4]:
%%f
: .chat // ( chat turn# -- ) Beautifully print the specified chat turn with markdown rendering.
    1- display Markdown
    <py>
    display, Markdown = pop(1), pop()
    import re
    sections = re.split("---===---", dictate('chat_history').pop())
    display(Markdown(sections[pop()]))
    </py> ;
    ///  0 最後 ---===--- 之後的 null。
    ///  1 第一個 chat turn。
    /// -1 最後一個 chat turn。


### 採用 OpenAI 

In [ ]:
# 查看 api key 採用 columbus if is null, otherwise 用你自己的 account.
os.getenv("OPENAI_API_KEY")

In [5]:
llm = columbus.get_llm_for_LangChain(openai_api_key="")
%f get_ipython to @get_ipython \ 把 get_ipython 介紹給 peforth
%f llm constant llm_object \ 把 llm 介紹給 peforth
%f : llm_wrapper ( prompt -- complete ) trim llm_object :> invoke(pop()).content ; \ 定義 llm_wrapper
%f ' llm_wrapper to @llm \ 把 llm_wrapper 介紹給 peforth 

### 採用 Gemini

In [ ]:
# 確定有 api key 
os.getenv("GoogleAIStudio_API_KEY")

In [ ]:
llm = columbus.get_llm_gemini(modelname="gemini-pro:generateContent", api_key=os.getenv("GoogleAIStudio_API_KEY"))
# llm = columbus.get_llm_gemini(api_key=os.getenv("GoogleAIStudio_API_KEY"))
%f get_ipython to @get_ipython \ 把 get_ipython 介紹給 peforth
%f llm constant llm_function \ 把 llm 介紹給 peforth
%f : llm_wrapper ( prompt -- complete ) trim llm_function :> (pop()) dup str :> rfind("candidates")==-1 if str else :> ["candidates"][0]['content']['parts'][0]['text'] then ;
%f ' llm_wrapper to @llm \ 把 llm_wrapper 介紹給 peforth 

### 採用 Antropic Claude

- [API reverence - SDK](https://docs.anthropic.com/en/api/client-sdks)
- [Models the prices](https://docs.anthropic.com/en/docs/about-claude/models#legacy-model-comparison)

In [ ]:
# 確定有 api key 
os.getenv("ANTHROPIC_API_KEY")

In [ ]:
!pip install anthropic
import anthropic
client = anthropic.Anthropic()
def llm(prompt):
    message = client.messages.create(
        model="claude-3-haiku-20240307", # $0.25 / $1.25 MTok
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return message

%f get_ipython to @get_ipython \ 把 get_ipython 介紹給 peforth
%f llm constant llm_function \ 把 llm 介紹給 peforth
%f : llm_wrapper ( prompt -- complete ) trim llm_function :> (pop()) dup str :> rfind("type='message'")==-1 if str else :> content[0].text then ;
%f ' llm_wrapper to @llm \ 把 llm_wrapper 介紹給 peforth 

## 試用 magics %ai and %chat 

In [16]:
%f "" to chat_history \ 把以前的 chatting contexts 都清除掉。
%ai tell me a joke 
%chat why is it funny ? 
%f -2 .chat -1 .chat

Why did the tomato turn red?

Because it saw the salad dressing!

Chat#16 2024-06-27 11:48:01

It's a play on words and a bit of a pun. The tomato turned red because it's a natural reaction when it ripens, but the joke implies that it turned red because it was embarrassed or blushing after seeing the salad dressing. It's a silly and unexpected twist that makes it humorous.

Chat#16 2024-06-27 11:48:01



**User:** tell me a joke

**Assistant:** Why did the tomato turn red?

Because it saw the salad dressing!

Chat#16 2024-06-27 11:48:01



**User:** why is it funny ?

**Assistant:** It's a play on words and a bit of a pun. The tomato turned red because it's a natural reaction when it ripens, but the joke implies that it turned red because it was embarrassed or blushing after seeing the salad dressing. It's a silly and unexpected twist that makes it humorous.

Chat#16 2024-06-27 11:48:01


## 查看 chat_history

In [7]:
%f chat_history . \ 最後有個小點，它是 forth 的 print 命令。

**User:** tell me a joke

**Assistant:** Why did the tomato turn red?

Because it saw the salad dressing!

Chat#6 2024-06-27 11:39:42
---===---
**User:** why is it funny ?

**Assistant:** It's a play on words and a bit of a pun. The tomato turned red because it's a natural reaction when it ripens, but the joke implies that it turned red because it was embarrassed or blushing after seeing the salad dressing. It's a silly and unexpected twist that makes it humorous.

Chat#6 2024-06-27 11:39:43
---===---


## 篡改 chat_history

In [8]:
%%f
<text>
User: Write me a poem.
Assistnat: What peom?
User: peforth new magic %ai and %chat for Jupyter notebook is now available.
</text> to chat_history

In [9]:
%chat 

In the world of coding, a new magic has arrived,
A tool for Jupyter, to help us thrive.
%ai and %chat, the names of the game,
Bringing new possibilities, never the same.

With AI at our fingertips, we can explore,
New ways of thinking, and so much more.
Chatting with machines, a futuristic dream,
Now a reality, or so it would seem.

The power of technology, in our hands,
A new era of coding, now expands.
Thanks to peforth, we can now create,
Innovative solutions, that are truly great.

So let's embrace this new magic, with open arms,
And see where it takes us, beyond the norms.
For the future is bright, with endless potential,
Thanks to peforth, and its new credentials.

Chat#9 2024-06-27 11:40:10



# Playground

In [11]:
%ai Tell me about 愛沙尼亞
%chat Their communist experience.  

愛沙尼亞 (Ài shā ní yà) is the Chinese name for Estonia, a small country located in Northern Europe. It is bordered by Russia to the east, Latvia to the south, and the Baltic Sea to the west and north.

Estonia has a population of approximately 1.3 million people and its capital city is Tallinn. The official language is Estonian, which is closely related to Finnish.

Estonia has a rich history, having been ruled by various powers throughout the centuries, including the Danes, Germans, Swedes, and Russians. It gained independence from the Soviet Union in 1991 and has since become a member of the European Union, NATO, and the United Nations.

Estonia is known for its beautiful natural landscapes, including forests, lakes, and islands. It is also a leader in technology, with a highly developed digital infrastructure and a thriving startup scene.

Some famous Estonians include composer Arvo Pärt, writer Jaan Kross, and former president Toomas Hendrik Ilves.

Chat#11 2024-06-27 11:41:02

**Assi

In [13]:
%f -2 .chat


**User:** Tell me about 愛沙尼亞

**Assistant:** 愛沙尼亞 (Ài shā ní yà) is the Chinese name for Estonia, a small country located in Northern Europe. It is bordered by Russia to the east, Latvia to the south, and the Baltic Sea to the west and north.

Estonia has a population of approximately 1.3 million people and its capital city is Tallinn. The official language is Estonian, which is closely related to Finnish.

Estonia has a rich history, having been ruled by various powers throughout the centuries, including the Danes, Germans, Swedes, and Russians. It gained independence from the Soviet Union in 1991 and has since become a member of the European Union, NATO, and the United Nations.

Estonia is known for its beautiful natural landscapes, including forests, lakes, and islands. It is also a leader in technology, with a highly developed digital infrastructure and a thriving startup scene.

Some famous Estonians include composer Arvo Pärt, writer Jaan Kross, and former president Toomas Hendrik Ilves.

Chat#11 2024-06-27 11:41:02


In [14]:
%f -1 .chat


**User:** Their communist experience.

**Assistant:** **Assistant:** Estonia was under Soviet rule from 1940 to 1991, during which time it experienced significant political and economic changes. The Soviet government implemented collectivization policies, which forced farmers to give up their land and join collective farms. This led to a decline in agricultural productivity and a shortage of food.

The Soviet government also established a planned economy, which meant that the government controlled all aspects of the economy, including production, distribution, and pricing. This led to a lack of innovation and inefficiencies in the economy.

During the Soviet era, Estonia also experienced political repression, with the government suppressing dissent and limiting freedom of speech and assembly. Many Estonians were sent to labor camps or executed for opposing the Soviet regime.

After gaining independence in 1991, Estonia transitioned to a market economy and implemented political reforms to establish a democratic government. Today, Estonia is a member of the European Union and NATO, and has a thriving economy and democratic political system.

Chat#11 2024-06-27 11:41:04


# The END